# Twitter Goggles Lite

Query the the Twitter Search API and return a structured table of results.

In [1]:
import numpy as np 
import pandas as pd

import argparse, collections, configparser, io, json, math, mysql.connector as sql, os, requests, sys, time
from datetime import datetime
from mysql.connector import errorcode
from requests import HTTPError
from requests import ConnectionError
from requests_oauthlib import OAuth1

The following script queries Twitter for tweets containing the name of 50 US cities in the body of the text as well as tweets withing a 40 km radius of that city. The 50 cities are listed within the `description` column of the `jobs.csv` file below and the query request are in the `query` column. 

In [2]:
job = pd.read_csv('data/job.csv')
job.head()

,job_id,state,zombie_head,since_id_str,query,description,last_count,last_run,analysis_state,oauth_id
0,204,23,1,8.541130e+17,q=Warren,"Warren, Michigan",511,4/17/17 18:23,0,4
1,205,24,2,8.541160e+17,q=Fresno,"Fresno, California",87,4/17/17 18:37,0,5
2,206,25,3,8.541140e+17,q=Cincinnati,"Cincinnati, Ohio",113,4/17/17 18:25,0,6
3,207,26,4,8.541170e+17,q=Oklahoma%20City,"Oklahoma City, Oklahoma",50,4/17/17 18:38,0,7
4,208,27,5,8.541150e+17,q=Allentown,"Allentown, Pennsylvania",23,4/17/17 18:33,0,8


In order to use the Search API, Twitter requires you to have authetication. Be sure to fill in the variables below with your Twitter creds, then run the cell.

In [3]:
consumer_key = 'H0pa8ePaLE9ZjRJp0BfFi5JVt'
consumer_secret = 'u2w1U1JccIrd5Zhy8O9ALPgdt1Z3xPyD3kJP8E8Q2AGde0jXpv'
access_token = '2882452026-qkvdVWqbNwJYktehDXlcensgRK6lZJG6cyWMPXm'
access_token_secret = '2V8jupOOtRYoNIkRLNewxc06vAuStj2ZcWAKa3fsZZUBw'


oauth = OAuth1(client_key=consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=access_token,
    resource_owner_secret=access_token_secret)

To make a request to Twitter, we define a function that queries Twitter and returns a json object that we will then parse into dictionaries for the attributes that we are interested in.

In [4]:
def search(query, oauth) :
    print("Query: " + query)
    
    attempt = 1
    while attempt <= 3 :
        try :
            r = requests.get("https://api.twitter.com/1.1/search/tweets.json?" + query, auth=oauth)
            return json.loads(r.text)
        
        except (ConnectionError, HTTPError) as err :
            sleep_time = 2**(attempt - 1)
            print("Connection attempt " + str(attempt) + " failed. "
                "Sleeping for " + str(sleep_time) + " second(s).")
            time.sleep(sleep_time)
            attempt = attempt + 1
    
    print("***** Error: Unable to query Twitter. Terminating.")

The function below just ensures that our query begins with "q=" as this is the format twitter requires. All of the jobs in the job table already have a "q=" but this acts as just a fail-safe. 

In [5]:
def getFullQuery(query) :
    if (not query.startswith("q=")) :
        query = "q=" + query
    return query

And now we structure our data! 

In [6]:
def structureTweetTable(tweet, job):
   
    list_o_dicts = []
    
    for i in tweet['statuses']:
        d = {}
        d['tweet_id_str'] =  i["id_str"] 
        d['job_id'] = job
        d['created_at'] = datetime.strptime(i["created_at"], '%a %b %d %H:%M:%S +0000 %Y').strftime('%Y-%m-%d %H:%M:%S')
        d['text'] = i["text"]
        d['from_user'] = i["user"]["id"]
        d['from_user_id_str'] = i["user"]["id_str"]
        d['from_user_name']  = i["user"]["screen_name"]
        d['from_user_fullname'] = i["user"]["name"]
        d['from_user_created_at'] = datetime.strptime(i["user"]["created_at"], '%a %b %d %H:%M:%S +0000 %Y').strftime('%Y-%m-%d %H:%M:%S')
        d['from_user_followers'] = i["user"]["followers_count"]
        d['from_user_following'] = i["user"]["friends_count"]
        d['from_user_favorites'] = i["user"]["favourites_count"]
        d['from_user_tweets'] = i["user"]["statuses_count"]
        d['from_user_timezone'] = i["user"]["time_zone"]
        d['to_user'] = i["in_reply_to_user_id"]
        d['to_user_id_str'] = i["in_reply_to_user_id_str"]
        d['to_user_name'] = i["in_reply_to_screen_name"]
        d['source'] = i["source"]
        d['iso_language'] = i["metadata"]["iso_language_code"]
        if i['geo'] is not None and i['geo']['type'] == "Point" :
            d['location_geo'] = 'Point({},{})'.format(i['geo']["coordinates"][0],i['geo']["coordinates"][1])
            d['location_geo_0'] = '{}'.format(i['geo']["coordinates"][0])
            d['location_geo_1'] = '{}'.format(i['geo']["coordinates"][1])
        
        list_o_dicts.append(d)

    return list_o_dicts
        

And now we put it all together and collect tweets from each job once. Each job run becomes a data frame that will then be appended to the list object `frames`.

In [7]:
frames = []
for index, row in job.iterrows():
    
    job_id = row['job_id']

    q = getFullQuery(row['query'])
    results = search(q,oauth)
    try:
        f = pd.DataFrame(structureTweetTable(results,job_id))
        frames.append(f)
    except:
        print("   no results returned for '{}'".format(row['query']))
        pass

    

Query: q=Warren
Query: q=Fresno
Query: q=Cincinnati
Query: q=Oklahoma%20City
Query: q=Allentown
Query: q=Columbia
Query: q=Cleveland
Query: q=Des%20Moines
Query: q=Grand%20Rapids
Query: q=Rochester
Query: q=Peoria
Query: q=Springfield
Query: q=Lafayette
Query: q=Overland%20Park
Query: q=Milwaukee
Query: q=Henderson
Query: q=Charlotte
Query: q=Chicago
Query: q=Brownsville
Query: q=Boston
Query: q=Baton%20Rouge
Query: q=Chandler
Query: q=Reno
Query: q=Santa%20Ana
Query: q=New%20Haven
Query: q=Broken%20Arrow
Query: q=Edison
Query: q=Rockford
Query: q=Philadelphia
Query: q=West%20Jordan
Query: q=Winston%20Salem
Query: q=Shreveport
Query: q=El%20Paso
Query: q=Birmingham
Query: q=Buffalo
Query: q=New%20York%20City
Query: q=Evansville
Query: q=Charleston
Query: q=Saint%20Paul
Query: q=Gresham
Query: q=Alexandria
Query: q=Houston
Query: q=Peoria
Query: q=North%20Charleston
Query: q=Provo
Query: q=Norman
Query: q=Augusta
Query: q=Seattle
Query: q=Durham
Query: q=Indianapolis
Query: q=st%20paul


In [8]:
frame = pd.concat(frames)

# Your turn...

From the data collected, create the following tables:
1. a table of hashtags
    - collect the hashtagged texts from each of the tweets
2. a table of mentions
    - collect all of the mentions ("@username") from each tweet
3. a table of urls
    - collect all of the urls from each tweet
    

Remember that you are structuring a database, so be sure to carry over the proper attributes in order to construct relations between other tables.

In [34]:
# Your Code (create as many cells as you need)
# --------------------------------------------
alls = []
for i, row in frame.iterrows():
    stuff = {tag.strip("#") for tag in row['text'].split() if tag.startswith("#")}
    if len(stuff) > 0:
        tweets = [[i] * len(stuff)]
        jobs = [[row['job_id']] * len(stuff)]
        f = pd.DataFrame({'tweet_id' : tweets[0], 'hastag' : list(stuff), 'job_id': jobs[0]})
        alls.append(f)
    



In [35]:
pd.concat(alls).reset_index().drop('index', axis = 1)

,hastag,job_id,tweet_id
0,giveaway,204,4
1,travel,204,4
2,Trump,204,7
3,Democrats,204,7
4,Election2020,204,7
5,ElizabethWarren,204,7
6,evans…,204,12
7,Black,205,0
8,jobs,205,6
9,Indeed,205,6


In [28]:
frame

,created_at,from_user,from_user_created_at,from_user_favorites,from_user_followers,from_user_following,from_user_fullname,from_user_id_str,from_user_name,from_user_timezone,...,job_id,location_geo,location_geo_0,location_geo_1,source,text,to_user,to_user_id_str,to_user_name,tweet_id_str
0,2017-04-24 16:42:03,396702061,2011-10-23 17:08:54,11030,3211,3801,Melanie Jackson,396702061,MelanieJaxn,Pacific Time (US & Canada),...,204,NaN,NaN,NaN,"<a href=""http://www.facebook.com/twitter"" rel=...","Happy birthday, Robert Penn Warren https://t.c...",NaN,None,None,856548805222604802
1,2017-04-24 16:42:01,2715717762,2014-08-08 00:54:17,14,16,232,Mary Kendall Hope,2715717762,MaryKendallHope,Eastern Time (US & Canada),...,204,NaN,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","Todd Warren, President GCAE will be my guest L...",NaN,None,None,856548793839472640
2,2017-04-24 16:41:58,22144947,2009-02-27 14:34:36,18986,130,349,sheri t.,22144947,sheeshkabob,Eastern Time (US & Canada),...,204,NaN,NaN,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",RT @thehill: Warren's public schedule looks li...,NaN,None,None,856548781197856769
3,2017-04-24 16:41:47,23019548,2009-03-06 02:49:25,4409,57,114,Ethel Cordova,23019548,Earthy007,Atlantic Time (Canada),...,204,NaN,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@AdamParkhomenko Elizabeth Warren. They didnt ...,1.83822e+07,18382184,AdamParkhomenko,856548738478858241
4,2017-04-24 16:41:46,151405084,2010-06-03 09:58:30,36,128590,44028,Warren J. Devalier,151405084,devalier_warren,Tokyo,...,204,NaN,NaN,NaN,"<a href=""http://paper.li"" rel=""nofollow"">Paper...",The latest Sports Fest! https://t.co/lFKhCyru6...,NaN,None,None,856548731818319872
5,2017-04-24 16:41:36,4724224163,2016-01-06 11:34:58,2858,42,63,ConradBFX,4724224163,ConradBfx,Pacific Time (US & Canada),...,204,NaN,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@lesleyabravanel @GovHowardDean Elizabeth Warr...,2.7943e+07,27943005,lesleyabravanel,856548689527136257
6,2017-04-24 16:41:34,852867577730695169,2017-04-14 12:54:10,6,2,22,Philosophical14God,852867577730695169,God14Peace,None,...,204,NaN,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","Ur Army Liz Warren has no basis in legal fact,...",NaN,None,None,856548684074496003
7,2017-04-24 16:41:34,2271670922,2014-01-01 13:57:48,977,27,445,Tyler,2271670922,V_I_uMsic,Pacific Time (US & Canada),...,204,NaN,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @opslens: Warren Builds Her Brand with 2020...,NaN,None,None,856548680891027457
8,2017-04-24 16:41:33,2577053906,2014-06-19 15:29:25,3290,1787,2149,Luigi Warren,2577053906,luigi_warren,Pacific Time (US & Canada),...,204,NaN,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @HWarlow: Politically incorrect these days ...,NaN,None,None,856548679263567872
9,2017-04-24 16:41:31,3487132335,2015-08-30 03:59:13,6927,619,471,abigail grammer,3487132335,abbgramm,Pacific Time (US & Canada),...,204,NaN,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...","RT @MasonCometsYL: This is a sad video, but Ma...",NaN,None,None,856548668605902848
